# Find Student Acommodation in Melbourne - For International Students
With several campuses scattered throughout the central business district, RMIT is surrounded by bustling city streets, cafés, entertainment venues and public transport. Enrolled at RMIT and want to live close to campus?

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
from csv import writer

!pip install geopy
!pip install folium

import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

# run a python file 
%run ../src/cleaning.py

# RMIT Melbourne Address - Geocoding Single Address

+ address: 124 La Trobe St, Melbourne VIC 3000, Australia
+ Unilodge Carlton: 9 min walk
+ Unilodge Victoria: about 24 min public transport
+ Unilodge Lincoln: 9 min walk
+ Dwell: 5 min tram, 15 min walk

In [2]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("124 La Trobe St, Melbourne VIC 3000, Australia")

print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

La Trobe Street, Melbourne Innovation District, Melbourne, City of Melbourne, Victoria, 3000, Australia
Latitude = -37.80874, Longitude = 144.966074


# [DWELL MELBOURNE STUDENT LIVING](https://www.dwellstudent.com.au/melbourne/#room)  (15 mins walk)

In [3]:
url= "https://www.dwellstudent.com.au/melbourne/#rooms"
page = requests.get(url)

scrape = soup(page.content, 'html.parser')
lists = scrape.find_all('div', class_="col-md-6 item")

with open('housing.csv', 'w', encoding='utf8', newline='') as f:
    thewriter = writer(f)
    header = ['title', 'icons', 'price', 'image']
    thewriter.writerow(header)

    for list in lists:
        title = list.find('div', class_="title")
        icons = list.find('div', class_="icon-number") #living, bed, bath numbers
        price = list.find('div', class_="price-wrapper")
        image = list.find('div', class_="img bg-cover-center")
        
        info = [title, icons, price, image]
        thewriter.writerow(info)
        
print(info)

[<div class="title">1 Bedroom Apartment</div>, <div class="icon-number">
<div class="icon living">1</div>
<div class="icon bed">1</div>
<div class="icon bath">1</div>
</div>, <div class="price-wrapper">
<div class="text">From</div>
<div class="price">
                    $529                  </div>
</div>, <div class="img bg-cover-center" style="background-image: url(https://www.dwellstudent.com.au/wp-content/uploads/2019/10/1-Br-Apartment-livingroom-h-768x512.jpg);">
<img class="img-mask" src="https://www.dwellstudent.com.au/wp-content/uploads/2019/10/1-Br-Apartment-livingroom-h-768x512.jpg"/>
</div>]


In [4]:
df = pd.read_csv('housing.csv')
df.head(3)

,title,icons,price,image
0,"<div class=""title"">5 Bedroom Apartment</div>","<div class=""icon-number"">\n<div class=""icon li...","<div class=""price-wrapper"">\n<div class=""text""...","<div class=""img bg-cover-center"" style=""backgr..."
1,"<div class=""title"">5 Bedroom Apartment Deluxe<...","<div class=""icon-number"">\n<div class=""icon li...","<div class=""price-wrapper"">\n<div class=""text""...","<div class=""img bg-cover-center"" style=""backgr..."
2,"<div class=""title"">4 Bedroom Apartment Deluxe<...","<div class=""icon-number"">\n<div class=""icon li...","<div class=""price-wrapper"">\n<div class=""text""...","<div class=""img bg-cover-center"" style=""backgr..."


In [5]:
import re
df['title'] = df['title'].str.replace(r'<[^<>]*>', '', regex=True)
df['icons'] = df['icons'].str.replace(r'<[^<>]*>', '', regex=True)
df['price'] = df['price'].str.replace(r'<[^<>]*>', '', regex=True).replace('\n', '', regex=True).replace(' ', '', regex=True)
df['image'] = df['image'].replace('<div class="img bg-cover-center" style="background-image: url\(', '', regex=True).replace('"/>\n</div>', '', regex=True).replace('\);">\n<img class="img-mask" src="', '', regex=True)

# added static columns inside the dataframe about Dwell Viallgae in Melbourne location
df['Name'] = 'Dwell'
df['address'] = '5-17 Flemington Road, North Melbourne, VIC – 3051, Australia'
df['link'] = 'https://www.dwellstudent.com.au/melbourne/#rooms'
df['transportation'] = '5 min tram, 15 min walk'

# Extract Last n characters from right of the column in pandas:
df['price'] = df['price'].str[-3:]

# split icons column by space into 3 different cols and delete icons column from the dataframe
df = df.join(df.pop('icons').str.split(expand=True))
#rename specific column names
df = df.rename(columns = {0:'living', 1:'bed', 2:'bath'})

df.head(3)

,title,price,image,address,link,transportation,living,bed,bath
0,5 Bedroom Apartment,285,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,5,2
1,5 Bedroom Apartment Deluxe,355,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,5,2
2,4 Bedroom Apartment Deluxe,339,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",4,4,2


In [6]:
df.price.unique

<bound method Series.unique of 0     285
1     355
2     339
3     309
4     387
5     298
6     229
7     329
8     439
9     439
10    459
11    479
12    529
Name: price, dtype: object>

# Converting addresses to geographic information — Latitude and Longitude

In [7]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent="scrape")

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['address'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df.head()

,title,price,image,address,link,transportation,living,bed,bath,location,point
0,5 Bedroom Apartment,285,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,5,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)"
1,5 Bedroom Apartment Deluxe,355,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,5,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)"
2,4 Bedroom Apartment Deluxe,339,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",4,4,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)"
3,3 Bedroom Apartment,309,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,3,1,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)"
4,3 Bedroom Apartment Large,387,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,3,1,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)"


In [8]:
print(df.columns)
# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head()

Index(['title', 'price', 'image', 'address', 'link', 'transportation',
       'living', 'bed', 'bath', 'location', 'point'],
      dtype='object')


,title,price,image,address,link,transportation,living,bed,bath,location,point,latitude,longitude,altitude
0,5 Bedroom Apartment,285,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,5,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)",-37.796671,144.951179,0.0
1,5 Bedroom Apartment Deluxe,355,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,5,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)",-37.796671,144.951179,0.0
2,4 Bedroom Apartment Deluxe,339,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",4,4,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)",-37.796671,144.951179,0.0
3,3 Bedroom Apartment,309,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,3,1,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)",-37.796671,144.951179,0.0
4,3 Bedroom Apartment Large,387,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 Flemington Road, North Melbourne, VIC – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,3,1,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)",-37.796671,144.951179,0.0


In [9]:
# drop unecessary columns
print(f'List of existing columns: {df.columns.to_list()}\n\n')
df = df.drop(['location', 'point', 'altitude', 'living'], axis=1)
print(f'List of existing columns after dropping: {df.columns.to_list()}\n\n')

List of existing columns: ['title', 'price', 'image', 'address', 'link', 'transportation', 'living', 'bed', 'bath', 'location', 'point', 'latitude', 'longitude', 'altitude']


List of existing columns after dropping: ['title', 'price', 'image', 'address', 'link', 'transportation', 'bed', 'bath', 'latitude', 'longitude']




In [10]:
print(f'NUMBER OF CATEGORIES: {df.price.nunique()}; \n\nUNIQUE NAMES OF THE CATEGORIES {df.price.unique()}\n\n\n')

NUMBER OF CATEGORIES: 12; 

UNIQUE NAMES OF THE CATEGORIES ['285' '355' '339' '309' '387' '298' '229' '329' '439' '459' '479' '529']





# Performe basic cleaning

In [11]:
"""
        Remove extra leading and tailing whitespace from the data.
        Lower case all contents inside the dataframe
"""

whitespace_remover(df)
# Cast all values inside the dataframe (except the columns' name) into lowercase
df = df.applymap(lambda s: s.lower() if type(s) == str else s)
df.to_csv('dwell.csv')
df

,title,price,image,address,link,transportation,bed,bath,latitude,longitude
0,5 bedroom apartment,285,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 flemington road, north melbourne, vic – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,2,-37.796671,144.951179
1,5 bedroom apartment deluxe,355,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 flemington road, north melbourne, vic – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,2,-37.796671,144.951179
2,4 bedroom apartment deluxe,339,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 flemington road, north melbourne, vic – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",4,2,-37.796671,144.951179
3,3 bedroom apartment,309,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 flemington road, north melbourne, vic – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,1,-37.796671,144.951179
4,3 bedroom apartment large,387,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 flemington road, north melbourne, vic – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,1,-37.796671,144.951179
5,2 bedroom studio,298,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 flemington road, north melbourne, vic – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",2,1,-37.796671,144.951179
6,2 bedroom studio with balcony,229,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 flemington road, north melbourne, vic – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",2,1,-37.796671,144.951179
7,2 bedroom apartment,329,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 flemington road, north melbourne, vic – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",2,1,-37.796671,144.951179
8,1 bedroom suite,439,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 flemington road, north melbourne, vic – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",1,1,-37.796671,144.951179
9,1 bedroom studio deluxe,439,https://www.dwellstudent.com.au/wp-content/upl...,"5-17 flemington road, north melbourne, vic – 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",1,1,-37.796671,144.951179


# [Unilodge Carlton](https://www.unilodge.com.au/student-accommodation-melbourne/rmit-bundoora) - 9 min walk
UniLodge @ RMIT Bundoora - Walert House, has everything you could possibly need. Located on the RMIT Bundoora Campus (West), this modern building offers affordable student accommodation with:

+ Spacious study rooms
+ A state-of-the-art cinema room
+ Outdoor BBQ area with herb garden
+ Gourmet kitchens
+ A pool table and games rooms
+ FREE laundry

### [MELBOURNE'S TALLEST STUDENT ACCOMMODATION TOWER ON ELIZABETH ST](https://www.unilodge.com.au/student-accommodation-melbourne/melbourne-city)
+ Common Area: Sky Lounge on L51 facilitated with dining and kitchenette.
+ Dining area with communal kitchen.
+ Cinema: Mini-Cinema with AV system and projector screen.
+ Music room: Music room provisioned with Grand Piano.

In [12]:
url= "https://www.unilodge.com.au/student-accommodation-melbourne/rmit-bundoora"
page = requests.get(url)

scrape = soup(page.content, 'html.parser')
lists = scrape.find_all('div', class_="ul_room")

with open('housing.csv', 'w', encoding='utf8', newline='') as f:
    thewriter = writer(f)
    header = ['title', 'price', 'image']
    thewriter.writerow(header)

    for list in lists:
        title = list.find('h3', class_="thumb__title h6")
        price = list.find('div', class_="LDSBestPrice-fromPrice")
        image = list.find('div', class_="thumb__body ratio-container")
        
        info = [title, price, image]
        thewriter.writerow(info)
        
print(info)

[None, None, None]


# [DOMAIN HOUSING](https://www.domain.com.au/sale/inspection-times/?ptype=apartment-unit-flat,block-of-units,duplex,free-standing,new-apartments,new-home-designs,new-house-land,pent-house,semi-detached,studio,terrace,town-house,villa&excludeunderoffer=1&keywords=rmit+university)

# [SCRAPE MELBOURNE HOUSING FOR RENT ON ZILLOW](https://www.zillow.com/melbourne-fl/rentals/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Melbourne%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-80.93409468457031%2C%22east%22%3A-80.39439131542969%2C%22south%22%3A27.96619499260209%2C%22north%22%3A28.335500912695093%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A19307%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D)